# Airavata Experiment SDK - Molecular Dynamics Example

This SDK allows users to define, plan, and execute molecular dynamics experiments with ease.
Here we demonstrate how to authenticate, set up a NAMD experiment, add replicas, create an execution plan, and monitor the execution.

## Install the required packages

First, install the `airavata-python-sdk-test` package from the pip repository.

In [ ]:
%pip install --upgrade airavata-python-sdk-test

## Import the Experiments SDK

In [2]:
import airavata_experiments as ae
from airavata_experiments import md

## Authenticate for Remote Execution

To authenticate for remote execution, call the `ae.login()` method.
This method will prompt you to enter your credentials and authenticate your session.

In [3]:
ae.login()

Using saved token


Once authenticated, the `ae.list_runtimes()` function can be called to list HPC resources that the user can access.

In [ ]:
runtimes = ae.list_runtimes()
display(runtimes)

## Upload Experiment Files

Drag and drop experiment files onto the workspace that this notebook is run on.

```bash
(sh) $: tree data
data
├── b4pull.pdb
├── b4pull.restart.coor
├── b4pull.restart.vel
├── b4pull.restart.xsc
├── par_all36_water.prm
├── par_all36m_prot.prm
├── pull.conf
├── structure.pdb
└── structure.psf

1 directory, 9 files

```

## Define a NAMD Experiment

The `md.NAMD.initialize()` is used to define a NAMD experiment.
Here, provide the paths to the `.conf` file, the `.pdb` file, the `.psf` file, any optional files you want to run NAMD on.
You can preview the function definition through auto-completion.

```python
def initialize(
    name: str,
    config_file: str,
    pdb_file: str,
    psf_file: str,
    ffp_files: list[str],
    other_files: list[str] = [],
    parallelism: Literal['CPU', 'GPU'] = "CPU",
    num_replicas: int = 1
) -> Experiment[ExperimentApp]
```

In [ ]:
exp = md.NAMD.initialize(
    name="yasith_namd_experiment",
    config_file="data/pull.conf",
    pdb_file="data/structure.pdb",
    psf_file="data/structure.psf",
    ffp_files=[
      "data/par_all36_water.prm",
      "data/par_all36m_prot.prm"
    ],
    other_files=[
      "data/b4pull.pdb",
      "data/b4pull.restart.coor",
      "data/b4pull.restart.vel",
      "data/b4pull.restart.xsc",
    ],
    parallelism="GPU",
)

To add replica runs, simply call the `exp.add_replica()` function.
You can call the `add_replica()` function as many times as you want replicas.
Any optional resource constraint can be provided here.

In [ ]:
exp.add_replica()

## Create Execution Plan

Call the `exp.plan()` function to transform the experiment definition + replicas into a stateful execution plan.
This plan can be exported in JSON format and imported back.

In [ ]:
plan = exp.plan()  # this will create a plan for the experiment
plan.describe()  # this will describe the plan
plan.save_json("plan.json")  # save the plan state

## Execute the Plan

In [ ]:
plan = ae.load_plan("plan.json")
plan.launch()
plan.save_json("plan.json")  # save the plan state

## Option A - Wait for Completion

In [ ]:
plan = ae.load_plan("plan.json")
plan.describe()

In [8]:
plan = ae.load_plan("plan.json")
plan.join()

Output()

Interrupted by user.


## Option B - Terminate Execution

In [ ]:
plan = ae.load_plan("plan.json")
plan.stop()

## Option C - Monitor Files During Execution

Displaying the status and files generated by each replica (task)

In [9]:
plan = ae.load_plan("plan.json")
for task in plan.tasks:
    status = task.status()
    print(f"status={status}")
    files = task.files()
    print(f"files={files}")

status=ExperimentStatus(state=4, timeOfStateChange=1733386271725, reason='process  started', statusId='EXPERIMENT_STATE_e5b4246d-9d7c-41c7-8a03-df8292941518')


Exception: Agent not found

Displaying the intermediate results generated by each replica (task)

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd

for index, task in enumerate(plan.tasks):

    @cs.task_context(task)
    def visualize():
        data = pd.read_csv("data.csv")
        plt.figure(figsize=(8, 6))
        plt.plot(data["x"], data["y"], marker="o", linestyle="-", linewidth=2, markersize=6)
        plt.title(f"Plot for Replica {index} of {len(plan.tasks)}")

    visualize()